<a href="https://colab.research.google.com/github/MehrdadJannesar/CNN_AdaBoost/blob/master/Copy_of_Copy_of_Untitled12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Load dataset
from keras.datasets import cifar10
import keras
(train_images, train_labels),(test_images, test_labels) = cifar10.load_data()

In [105]:
#Dataset's information
print("train_images dimention: " , train_images.ndim)
print("train_images shape: ", train_images.shape)
print("train_image type: ", train_images.dtype)

train_images dimention:  4
train_images shape:  (50000, 32, 32, 3)
train_image type:  uint8


In [0]:
#Optimazation and categorical dataset
X_train = train_images.astype('float32')
X_test = test_images.astype('float32')


X_train /= 255
X_test /= 255


N_classes = 10
cifar_classes = ["airplane", "automobile", "bird", "cat", "deer", 
                   "dog", "frog", "horse", "ship", "truck"]

Y_train = keras.utils.to_categorical(train_labels,N_classes)
Y_test = keras.utils.to_categorical(test_labels,N_classes)

In [0]:
from keras.models import Model
from keras.layers import Dense,Input,Flatten
from keras.applications.resnet50 import ResNet50

In [108]:
#Connect to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
#Create model
input_layer = Input((32, 32, 3))
resnet = ResNet50(include_top = False, pooling = 'avg'
                  , weights = '/content/gdrive/My Drive/Data/Data/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')(input_layer)

dense = Dense(N_classes, activation = 'softmax')(resnet)

myModel = Model(input_layer, dense)


In [110]:
#Summary model
myModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
resnet50 (Model)             (None, 2048)              23587712  
_________________________________________________________________
dense_20 (Dense)             (None, 10)                20490     
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [0]:
#Initial optimizer
import keras
sgd = keras.optimizers.SGD(lr = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
myModel.compile(optimizer = sgd, loss = keras.losses.categorical_crossentropy, metrics = ['accuracy'])

In [0]:
#CallBacks
#Save model

myModel.save('/content/gdrive/My Drive/Data/Data/model{epoch}_func_api.h5')
myModel.save_weights('/content/gdrive/My Drive/Data/Data/model_weights_func_api.h5')

from keras import callbacks


model_checkpoint = callbacks.ModelCheckpoint('/content/gdrive/My Drive/Data/Data/model_func_api.h5')
model_log = callbacks.CSVLogger('/content/gdrive/My Drive/Data/Data/model_func_api.log')
model_tensor = callbacks.TensorBoard('/content/gdrive/My Drive/Data/Data/Tensorboard/run2/_func_api ')
callbacks= [model_checkpoint, model_log, model_tensor]

In [0]:
#Load model

# from keras.models import load_model

# model_load = load_model('/content/gdrive/My Drive/Data/Data/model{epoch}.h5')


In [113]:
#Learning network
import datetime

start = datetime.datetime.now()

network_history = myModel.fit(X_train, Y_train, batch_size=128, epochs=50 ,validation_split=0.2
                             ,callbacks=callbacks)

end = datetime.datetime.now()

print('time compile is : ', str(end - start))

Train on 40000 samples, validate on 10000 samples
Epoch 1/50
40000/40000 [==============================] - 35s 866us/step - loss: 1.4898 - acc: 0.5246 - val_loss: 1.4553 - val_acc: 0.5345
Epoch 2/50
40000/40000 [==============================] - 29s 735us/step - loss: 1.5752 - acc: 0.4694 - val_loss: 3.0558 - val_acc: 0.1940
Epoch 3/50
40000/40000 [==============================] - 30s 743us/step - loss: 1.8124 - acc: 0.3730 - val_loss: 12.3485 - val_acc: 0.1078
Epoch 4/50
40000/40000 [==============================] - 30s 746us/step - loss: 1.5182 - acc: 0.4515 - val_loss: 1.4560 - val_acc: 0.4865
Epoch 5/50
40000/40000 [==============================] - 30s 752us/step - loss: 1.3884 - acc: 0.5061 - val_loss: 1.3821 - val_acc: 0.5140
Epoch 6/50
40000/40000 [==============================] - 30s 751us/step - loss: 1.2437 - acc: 0.5538 - val_loss: 1.3705 - val_acc: 0.5153
Epoch 7/50
40000/40000 [==============================] - 30s 749us/step - loss: 1.1232 - acc: 0.5971 - val_loss: 1

In [116]:
input_ress = myModel.get_layer(index=1).inputs
output_ress = myModel.get_layer(index=1).outputs
ress_model = Model(inputs=input_ress,outputs=output_ress)

ress_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [114]:
#Predict network
test_loss, test_acc = myModel.evaluate(X_test, Y_test)
test_prediction = myModel.predict(X_test)


print("test_loss: ",test_loss)
print("test_acc: ",test_acc)
print("test_prediction", test_prediction)

10000/10000 [==============================] - 4s 395us/step
test_loss:  1.9680618893623352
test_acc:  0.6961
test_prediction [[3.4040205e-08 1.6354777e-07 9.5380104e-10 ... 1.6873090e-08
  9.6661051e-07 1.0102168e-06]
 [1.2266155e-07 3.7930761e-07 2.2784805e-11 ... 3.8310423e-12
  9.9999952e-01 1.3964484e-08]
 [8.8844541e-07 6.7502365e-10 2.2170652e-09 ... 2.0003104e-06
  9.9991274e-01 8.4349929e-05]
 ...
 [6.8143829e-10 1.2649939e-11 5.7527867e-09 ... 7.8252897e-09
  1.0659274e-10 1.1147771e-10]
 [9.7360826e-01 7.8385938e-03 1.1258654e-05 ... 5.5821547e-05
  1.0079679e-05 2.4307610e-07]
 [2.0746176e-09 4.5231867e-09 1.7049033e-06 ... 9.9848986e-01
  9.1992669e-10 2.7769145e-08]]


In [0]:
import numpy as np

#Adaboost_Layer_1
input_ress = myModel.get_layer(index=1).inputs
output_ress = myModel.get_layer('resnet50').get_layer('add_29').output
dens_adaboostLayer_1 = Dense(N_classes,activation='softmax')(output_ress)
adaboostLayer_1 = Model(inputs=input_ress,outputs=dens_adaboostLayer_1)
predict_adaLayer_1 = adaboostLayer_1.predict(X_test)
predict_test_uniq_1 = np.argmax(predict_adaLayer_1, axis=1)
predict_test_uniq_1 = predict_test_uniq_1[:,0,:]

#Adaboost_Layer_2
input_ress = myModel.get_layer(index=1).inputs
output_ress = myModel.get_layer('resnet50').get_layer('add_30').output
dens_adaboostLayer_2 = Dense(N_classes,activation='softmax')(output_ress)
adaboostLayer_2 = Model(inputs=input_ress,outputs=dens_adaboostLayer_2)
predict_adaLayer_2 = adaboostLayer_2.predict(X_test)
predict_test_uniq_2 = np.argmax(predict_adaLayer_2, axis=1)
predict_test_uniq_2 = predict_test_uniq_2[:,0,:]

#Adaboost_Layer_3
input_ress = myModel.get_layer(index=1).inputs
output_ress = myModel.get_layer('resnet50').get_layer('add_31').output
densadaboostLayer_3 = Dense(N_classes,activation='softmax')(output_ress)
adaboostLayer_3 = Model(inputs=input_ress,outputs=densadaboostLayer_3)
predict_adaLayer_3 = adaboostLayer_3.predict(X_test)
predict_test_uniq_3 = np.argmax(predict_adaLayer_3, axis=1)
predict_test_uniq_3 = predict_test_uniq_3[:,0,:]

#Adaboost_Layer_4
input_ress = myModel.get_layer(index=1).inputs
output_ress = myModel.get_layer('resnet50').get_layer('add_32').output
dens_adaboostLayer_4 = Dense(N_classes,activation='softmax')(output_ress)
adaboostLayer_4 = Model(inputs=input_ress,outputs=dens_adaboostLayer_4)
predict_adaLayer_4 = adaboostLayer_4.predict(X_test)
predict_test_uniq_4 = np.argmax(predict_adaLayer_4, axis=1)
predict_test_uniq_4 = predict_test_uniq_4[:,0,:]

In [140]:
#Ensembel method (AdaBoost)
from sklearn.ensemble import AdaBoostClassifier

ada_boost_predict_1 = AdaBoostClassifier()
ada_boost_predict_2 = AdaBoostClassifier()
ada_boost_predict_3 = AdaBoostClassifier()
ada_boost_predict_4 = AdaBoostClassifier()

ada_boost_predict_1.fit(predict_test_uniq_1, test_labels)
ada_boost_predict_2.fit(predict_test_uniq_2, test_labels)
ada_boost_predict_3.fit(predict_test_uniq_3, test_labels)
ada_boost_predict_4.fit(predict_test_uniq_4, test_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)

In [0]:
#Feature Adaboost
feature_predict_1 = ada_boost_predict_1.feature_importances_  
feature_predict_2 = ada_boost_predict_2.feature_importances_  
feature_predict_3 = ada_boost_predict_3.feature_importances_  
feature_predict_4 = ada_boost_predict_4.feature_importances_  

In [0]:
#Predict Adaboost
ada_predict_1 = ada_boost_predict_1.predict(predict_test_uniq_1)
ada_predict_2 = ada_boost_predict_2.predict(predict_test_uniq_2)
ada_predict_3 = ada_boost_predict_3.predict(predict_test_uniq_3)
ada_predict_4 = ada_boost_predict_4.predict(predict_test_uniq_4)

In [144]:
#Score Adaboost
ada_predict_1_score = ada_boost_predict_1.score(predict_test_uniq_1, test_labels)
ada_predict_2_score = ada_boost_predict_2.score(predict_test_uniq_2, test_labels)
ada_predict_3_score = ada_boost_predict_3.score(predict_test_uniq_3, test_labels)
ada_predict_4_score = ada_boost_predict_4.score(predict_test_uniq_4, test_labels)

print("ada_predict_1_score: ",ada_predict_1_score)
print("ada_predict_2_score: ",ada_predict_2_score)
print("ada_predict_3_score: ",ada_predict_3_score)
print("ada_predict_4_score: ",ada_predict_4_score)

ada_predict_1_score:  0.2925
ada_predict_2_score:  0.1
ada_predict_3_score:  0.1
ada_predict_4_score:  0.1


In [0]:
#Multi-class AdaBoosted

from sklearn.metrics import accuracy_score

real_test_errors = []
discrete_test_errors = []
#predict_test_uniq_1, test_labels
for real_test_predict, discrete_train_predict in zip(
        ada_boost_predict_1.staged_predict(predict_test_uniq_1), ada_boost_predict_2.staged_predict(predict_test_uniq_2)):
    real_test_errors.append(
        1. - accuracy_score(real_test_predict, test_labels))
    discrete_test_errors.append(
        1. - accuracy_score(discrete_train_predict, test_labels))

    
    
    
n_trees_discrete = len(ada_boost_predict_2)
n_trees_real = len(ada_boost_predict_1)

# Boosting might terminate early, but the following arrays are always
# n_estimators long. We crop them to the actual number of trees here:
discrete_estimator_errors = ada_boost_predict_2.estimator_errors_[:n_trees_discrete]
real_estimator_errors = ada_boost_predict_1.estimator_errors_[:n_trees_real]
discrete_estimator_weights = ada_boost_predict_2.estimator_weights_[:n_trees_discrete]

In [0]:
real_test_errors_2 = []
discrete_test_errors_2 = []
#predict_test_uniq_1, test_labels
for real_test_predict, discrete_train_predict in zip(
        ada_boost_predict_3.staged_predict(predict_test_uniq_3), ada_boost_predict_4.staged_predict(predict_test_uniq_4)):
    real_test_errors_2.append(
        1. - accuracy_score(real_test_predict, test_labels))
    discrete_test_errors_2.append(
        1. - accuracy_score(discrete_train_predict, test_labels))

    
    
    
n_trees_discrete_2 = len(ada_boost_predict_4)
n_trees_real_2 = len(ada_boost_predict_3)

# Boosting might terminate early, but the following arrays are always
# n_estimators long. We crop them to the actual number of trees here:
discrete_estimator_errors_2 = ada_boost_predict_4.estimator_errors_[:n_trees_discrete_2]
real_estimator_errors_2 = ada_boost_predict_3.estimator_errors_[:n_trees_real_2]
discrete_estimator_weights_2 = ada_boost_predict_4.estimator_weights_[:n_trees_discrete_2]

In [185]:
discrete_estimator_errors_2

array([0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9,
       0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9])